In [1]:
import mysql.connector as sql


In [2]:
connection = sql.connect(
    host="localhost",
    user="root",
    password="Ayoub1111",
    allow_local_infile=True
)

In [4]:
cursor = connection.cursor()

In [5]:
cursor.execute("CREATE DATABASE IF NOT EXISTS product_sales")

In [6]:
cursor.execute('use product_sales')

In [7]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS discount (
        Month VARCHAR(255) NOT NULL,
        Discount_Band VARCHAR(255) NOT NULL,
        Discount INT NOT NULL
);
"""

cursor.execute(create_table_query)

In [8]:
connection.commit()

In [9]:
file_path = '/home/hp/Sql-Sales-Analysis/data/discount_data.csv'
cursor.execute(
    f"""
    LOAD DATA LOCAL INFILE '{file_path}'
    INTO TABLE discount
    FIELDS TERMINATED BY ','
    ENCLOSED BY '"'
    LINES TERMINATED BY '\n'
    IGNORE 1 LINES
    (Month, Discount_Band, Discount)
    """)


In [10]:
connection.commit()

In [11]:
cursor.execute("SHOW TABLES LIKE 'discount'")

In [12]:
result = cursor.fetchone()

In [13]:
if result:
    print("table exists")
else:
    print("table not exists")

table exists


In [14]:
table_name = 'discount'

In [15]:
query = f"SELECT * FROM {table_name}"

In [16]:
cursor.execute(query)
 

In [17]:
result = cursor.fetchall()

In [18]:
product_query = """
    CREATE TABLE IF NOT EXISTS Products (
        ProductID VARCHAR(255) NOT NULL,
        Product VARCHAR(255) NOT NULL,
        Category VARCHAR(255) NOT NULL,
        Cost_Price VARCHAR(255) NOT NULL,
        Sale_Price INT NOT NULL,
        Brand VARCHAR(255) NOT NULL,
        Description VARCHAR(255) NOT NULL
);
"""

In [19]:
cursor.execute(product_query)

In [20]:
connection.commit()

In [29]:
file_path = '../home/hp/Sql-Sales-Analysis/data/Product_data.csv'
cursor.execute(
    f"""
    LOAD DATA LOCAL INFILE '{file_path}'
    INTO TABLE Products
    FIELDS TERMINATED BY ','
    ENCLOSED BY '"'
    LINES TERMINATED BY '\n'
    IGNORE 1 LINES 
    (ProductID, Product, Category, Cost_Price, Sale_Price, Brand, Description)
    """)


In [30]:
connection.commit()

InterfaceError: File '/home/hp/Sql-Sales-Analysis/home/hp/Sql-Sales-Analysis/data/Product_data.csv' could not be read